[Assistants migration guide](https://platform.openai.com/docs/assistants/migration)

# Constants and Libraries

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv

load_dotenv("./../../config/credentials_my.env")
QUESTION="How many flights do we have between my cat born date and Easter 2021?"

# Create a client to connecto Azure OpenAI service and deployment

In [2]:
from openai import AzureOpenAI

# Create the client
client = AzureOpenAI(
    # api_key        = os.getenv("AZURE_OPENAI_API_KEY"),  
    # api_version    = os.getenv("AZURE_OPENAI_API_VERSION"), # at least 2024-02-15-preview
    # azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)
print(f"client.base_url: {client.base_url}")

client.base_url: https://mmoaiswc-01.openai.azure.com/openai/


# Define custom functions

In [3]:
def get_flights(date_1:str, date_2:str) -> dict:
    """ Returns the number of flights in a date interval  """
    import json
    from dateutil.parser import parse
    flights = {
        "flights": abs((parse(date_2) - parse(date_1)).days) 
    }
    return json.dumps(flights)

def my_cat_born_date() -> dict:
    """ Returns my cat's born date """
    import datetime, random, json
    from dateutil.relativedelta import relativedelta
    
    # Calculate the date as ten years ago  
    ten_years_ago = datetime.date.today() - relativedelta(years=10) 
    
    cat_born_date = {
        "cat_born_date": ten_years_ago.strftime("%Y-%m-%d")
    }
    return json.dumps(cat_born_date)

print(f'get_flights("2025-01-15", "2025-02-15") --> {get_flights("2025-01-15", "2025-02-15")}')
print(f'my_cat_born_date --> {my_cat_born_date()}')

get_flights("2025-01-15", "2025-02-15") --> {"flights": 31}
my_cat_born_date --> {"cat_born_date": "2015-01-29"}


# Wrap the functions into their tools list

In [4]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_flights",
      "description": "returns the number of flights between two dates",
      "parameters": {
        "type": "object",
        "properties": {
          "date_1": {
            "type": "string",
            "description": "the first date"
          },
          "date_2": {
            "type": "string",
            "description": "the second date"
          }
        },
        "required": [
          "date_1",
          "date_2"
        ]
      }
    }
  },
  {
    "type": "function",
    "function": {
      "name": "my_cat_born_date",
      "description": "returns my cat's born date",
      "parameters": {
        "type": "object",
        "properties": {},
        "required": []
      }
    }
  }
]

# Create an Assistant

In [5]:
assistant = client.beta.assistants.create(
    name="Smart Assistant",
    description="You are a helpful AI assistant who helps answering questions",
    tools = tools,
    model = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME")
)

# print(assistant.model_dump_json(indent=2))

# Create a conversation
Note that `code_interpreter` and `file_search` are empty in this case, because `code_interpreter` was not created.

In [6]:
# Create a thread
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_FNp1wDWnEJmSqFIyzGjrugVw', created_at=1738145481, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


# Add a message to the conversation

In [7]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=QUESTION
)

thread_messages = client.beta.threads.messages.list(thread.id)

print(thread_messages.model_dump_json(indent=2))

{
  "data": [
    {
      "id": "msg_Z2xekrNTN3JNIoeqbLqzg4Hu",
      "assistant_id": null,
      "attachments": [],
      "completed_at": null,
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "How many flights do we have between my cat born date and Easter 2021?"
          },
          "type": "text"
        }
      ],
      "created_at": 1738145481,
      "incomplete_at": null,
      "incomplete_details": null,
      "metadata": {},
      "object": "thread.message",
      "role": "user",
      "run_id": null,
      "status": null,
      "thread_id": "thread_FNp1wDWnEJmSqFIyzGjrugVw"
    }
  ],
  "object": "list",
  "first_id": "msg_Z2xekrNTN3JNIoeqbLqzg4Hu",
  "last_id": "msg_Z2xekrNTN3JNIoeqbLqzg4Hu",
  "has_more": false
}


# Create a Run and check its status

In [8]:
import time, json

run            = client.beta.threads.runs.create(
  thread_id    = thread.id,
  assistant_id = assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

while client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status in ["queued", "in_progress"]:
    print(f"Run status: {run.status}")
    time.sleep(5)

print(f"Final run status: {client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status}")

json.loads(client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).to_json())

Run status: queued
Final run status: requires_action


{'id': 'run_DkAo2JpvHCyZzmadIRJXGaxp',
 'assistant_id': 'asst_DsxCLlGtpvaGlWebi8ZWj5Av',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1738145482,
 'expires_at': 1738146082,
 'failed_at': None,
 'incomplete_details': None,
 'instructions': None,
 'last_error': None,
 'max_completion_tokens': None,
 'max_prompt_tokens': None,
 'metadata': {},
 'model': 'gpt-4o',
 'object': 'thread.run',
 'parallel_tool_calls': True,
 'required_action': {'submit_tool_outputs': {'tool_calls': [{'id': 'call_gYiMZMYUHPPr5NCqOs9chAiN',
     'function': {'arguments': '{}', 'name': 'my_cat_born_date'},
     'type': 'function'}]},
  'type': 'submit_tool_outputs'},
 'response_format': 'auto',
 'started_at': 1738145482,
 'status': 'requires_action',
 'thread_id': 'thread_FNp1wDWnEJmSqFIyzGjrugVw',
 'tool_choice': 'auto',
 'tools': [{'function': {'name': 'get_flights',
    'description': 'returns the number of flights between two dates',
    'parameters': {'type': 'object',
     'properties': {'date

# If `status` == `requires_action`, run the function(s) required

In [12]:
run_json = json.loads(client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).to_json())
if run_json["status"]=="requires_action":
    tool_outputs = []
    for tc in run_json['required_action']['submit_tool_outputs']['tool_calls']:
        tc_id = tc['id']
        tc_function_name = tc['function']['name']
        tc_args = tc['function']['arguments']
        tool_output = eval(tc_function_name)(**json.loads(tc_args))
        print(f"Calling {tc_function_name}({tc_args}) --> {tool_output}")
        tool_outputs.append({"tool_call_id": tc_id, "output": tool_output})
    print(f"tool_outputs: {tool_outputs}")
else:
    print(f'No more actions required. Run status: {run_json["status"]}')

Calling get_flights({"date_1":"2015-01-29","date_2":"2021-04-04"}) --> {"flights": 2257}
tool_outputs: [{'tool_call_id': 'call_VHiSRszDZQFZQKcOCD2pB4z5', 'output': '{"flights": 2257}'}]


# Pass result(s) of the executed function(s) back to the model via `tool_outputs`

In [13]:
if client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status == "completed":
    print("The run status is completed, you can move on and retrieve the final result ;-)")

else:
    run = client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        tool_outputs=tool_outputs,
        run_id=run.id
    )
    
    while client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status in ["queued", "in_progress"]:
        print(f"Run status: {run.status}")
        time.sleep(5)
    
    run_status = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status
    
    print(f"Final run status: {run_status}")
    
    if run_status == "requires_action":
        print("Please run again the required function(s) in the above cell and then the current cell, until the run is completed.")
    elif run_status == "completed":
        print("The run status is completed, you can move on and retrieve the final result ;-)")
    else:
        print(f"Please check why we have this run <{run_status}> status")

Run status: queued
Final run status: completed
The run status is completed, you can move on and retrieve the final result ;-)


In [ ]:
run

# Extract the final result from the run
## Run the next cells **ONLY IF** `run.status = completed`

In [14]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(json.loads(messages.to_json())["data"][0]["content"][0]["text"]["value"])

There are 2,257 flights between my cat's birth date (January 29, 2015) and Easter 2021 (April 4, 2021).


# Final implementation

In [ ]:
import time, json

# Create a thread
thread = client.beta.threads.create()

print (f"Question: {QUESTION}")

message       = client.beta.threads.messages.create(
    thread_id = thread.id,
    role      = "user",
    content   = QUESTION
)

run            = client.beta.threads.runs.create(
  thread_id    = thread.id,
  assistant_id = assistant.id,
  #instructions="New instructions" #You can optionally provide new instructions but these will override the default instructions
)

start_time = time.time()
status = run.status

while status not in ["completed", "cancelled", "expired", "failed"]:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id,run_id=run.id)
    print( f"Run status: {status} after {int((time.time() - start_time) // 60)} minutes {int((time.time() - start_time) % 60)} seconds")
    
    if status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []
        for tc in tool_calls:
            function_to_call = tc.function.name
            function_args    = tc.function.arguments
            tool_output      = eval(function_to_call)(**json.loads(function_args))
            tool_outputs.append({"tool_call_id": tc.id, "output": tool_output})

            print(f"Calling {tc.function.name}(**{tc.function.arguments}) --> {tool_output}")

        run = client.beta.threads.runs.submit_tool_outputs(
            thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs)

    time.sleep(5)
    status = run.status
        
print( f"\nRun status: {status} after {int((time.time() - start_time) // 60)} minutes {int((time.time() - start_time) % 60)} seconds")
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(json.loads(messages.to_json())["data"][0]["content"][0]["text"]["value"])